In [1]:
import pandas as pd
import json

from pathlib import Path
import numpy as np
import sys
import os

# Add project root to Python path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from src.config import config
from src.data.etl import load_data
from src.model.bert import TextEmbedder, DimensionalityReducer, ClusteringModel, CTFIDFVectorizer
from src.model.visualize import TopicVisualizer
from src.model.eval import TopicModelEvaluator
from src.model.label import LLMTopicNamer

from dotenv import load_dotenv

# os.environ['HF_HUB_OFFLINE'] = '1' # Force offline mode for HuggingFace
# os.environ['TRANSFORMERS_OFFLINE'] = '1' # Force offline mode for Transformers
load_dotenv()

/Users/Z00GK5Z/Documents/Workspace/Project/topic_modeling/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
from huggingface_hub import scan_cache_dir

cache_info = scan_cache_dir()
print("Cached models:")
for repo in cache_info.repos:
    print(f"  - {repo.repo_id} ({repo.size_on_disk_str})")
    
cache_dir = Path.home() / ".cache" / "huggingface" / "hub"
print(f"HuggingFace cache directory: {cache_dir}")
print(f"Exists: {cache_dir.exists()}")

Cached models:
  - BAAI/bge-small-en-v1.5 (134.5M)
  - cross-encoder/ms-marco-MiniLM-L-6-v2 (91.8M)
  - sentence-transformers/all-MiniLM-L6-v2 (91.6M)
  - unsloth/Qwen3-8B-GGUF (5.0G)
  - Qwen/Qwen3-4B (8.1G)
  - Qwen/Qwen3-Embedding-0.6B (1.2G)
  - Qwen/Qwen3-8B (16.4G)
  - Qwen/Qwen3-8B-AWQ (1.0K)
  - mlx-community/Qwen3-8B-4bit (939.0)
  - FacebookAI/roberta-base (2.7M)
  - BAAI/bge-large-en-v1.5 (1.3G)
HuggingFace cache directory: /Users/Z00GK5Z/.cache/huggingface/hub
Exists: True


In [3]:
print(config)

Settings Configuration:
  BASE_PATH: /Users/Z00GK5Z/Documents/Workspace/Project/topic_modeling
  DATA_FILE_PATH: /Users/Z00GK5Z/Documents/Workspace/Project/topic_modeling/data/raw/data.json
  SEED: 23
  LLM_CONFIG: {'model_name': 'Qwen/Qwen3-8B', 'max_new_tokens': 80, 'temperature': 0.1, 'use_case': 'customer_complaints'}
  LOG_CONFIG: {'version': 1, 'disable_existing_loggers': False, 'formatters': {'detailed': {'format': '%(asctime)s - %(name)s - %(levelname)s - %(message)s', 'datefmt': '%Y-%m-%d %H:%M:%S'}, 'simple': {'format': '%(levelname)s - %(message)s'}}, 'handlers': {'console': {'class': 'logging.StreamHandler', 'level': 'INFO', 'formatter': 'simple', 'stream': 'ext://sys.stdout'}, 'file': {'class': 'logging.handlers.RotatingFileHandler', 'level': 'DEBUG', 'formatter': 'detailed', 'filename': '/Users/Z00GK5Z/Documents/Workspace/Project/topic_modeling/output/topic_modeling.log', 'maxBytes': 10485760, 'backupCount': 5, 'encoding': 'utf-8'}}, 'root': {'level': 'INFO', 'handlers': 

In [4]:
data = load_data()

data.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4.0,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5.0,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5.0,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4.0,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5.0,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [5]:
data = data[data['asin'] == '6073894996']
docs = data['reviewText'].tolist()

print("Number of Docs:",len(docs))

print("Unique ASINs:",data['asin'].nunique())
print("Sample documents:")
docs[:8]

Number of Docs: 37
Unique ASINs: 1
Sample documents:


['Surprisingly, this inexpensive version works just as well and just as reliably as the expensive variety. It has been working for me for months now. No problem. Excellent value.',
 'I have tested this against the griffin dual output unit.I checked the charging current.This unit was charging my galaxy note battery with 70 ma.Griffin was charging with 40 ma! And the griffin was 4 times more expensive.I have not used these for very long. I bought 15 of them, because they are so cheap and because they actually do seem to provide high current.No idea how long they last. I assume they will work fine. I have not been using them much. I did the testing , just to stock up on a high current charger. This passed and I stocked up.',
 'It worked great for the first couple of weeks then it just stopped completely.. so basically a small waste of money.',
 'I love that it has two ports for my phone and ipod. Who wants to be putting too many things in one socket.  Sleek and convenient to store and I j

In [6]:
embedder = TextEmbedder(**config.embedding_model_config)
embeddings = embedder.embed(docs)

print("Embeddings shape:", embeddings.shape)

INFO - Use pytorch device_name: mps
INFO - Load pretrained SentenceTransformer: BAAI/bge-large-en-v1.5


'(ProtocolError('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')), '(Request ID: 20af6911-e432-4071-bcba-3b7bc24f8777)')' thrown while requesting HEAD https://huggingface.co/BAAI/bge-large-en-v1.5/resolve/main/./modules.json


WARNING - '(ProtocolError('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')), '(Request ID: 20af6911-e432-4071-bcba-3b7bc24f8777)')' thrown while requesting HEAD https://huggingface.co/BAAI/bge-large-en-v1.5/resolve/main/./modules.json


Retrying in 1s [Retry 1/5].


WARNING - Retrying in 1s [Retry 1/5].


...

WARNING - '(ProtocolError('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')), '(Request ID: 17b0181e-e671-4da0-aa9b-7bca7cd1c265)')' thrown while requesting HEAD https://huggingface.co/api/resolve-cache/models/BAAI/bge-large-en-v1.5/d4aa6901d3a41ba39fb536a557fa166f842b0e09/1_Pooling%2Fconfig.json


Retrying in 1s [Retry 1/5].


WARNING - Retrying in 1s [Retry 1/5].
Using device: Apple Silicon GPU (MPS)
  macOS: True


Batches: 100%|██████████| 5/5 [00:00<00:00,  9.85it/s]

Embeddings shape: (37, 1024)


In [7]:
reducer = DimensionalityReducer(**config.dr_config)
reduced_embeddings = reducer.fit_transform(embeddings)

INFO - Initialized UMAP with n_components=5
INFO - Reducing dimensionality from 1024 to 5 using UMAP


/Users/Z00GK5Z/Documents/Workspace/Project/topic_modeling/.venv/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


INFO - Dimensionality reduction complete: (37, 5)


In [8]:
cluster_model = ClusteringModel(**config.clustering_config)
clusters_labels = cluster_model.fit_predict(reduced_embeddings)

INFO - Initialized HDBSCAN (auto-discovers topics)
INFO - Clustering 37 documents using HDBSCAN
INFO - Clustering complete: Found 5 topics
INFO - Outliers/Noise: 4 documents (10.8%)


/Users/Z00GK5Z/Documents/Workspace/Project/topic_modeling/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/Z00GK5Z/Documents/Workspace/Project/topic_modeling/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [9]:
docs_per_topic = {}
for doc, label in zip(docs, clusters_labels):
    if label == -1:  
        continue
    if label not in docs_per_topic:
        docs_per_topic[label] = []
    docs_per_topic[label].append(doc)

topic_docs = []
topic_ids = []
for topic_id in sorted(docs_per_topic.keys()):
    topic_docs.append(' '.join(docs_per_topic[topic_id]))
    topic_ids.append(topic_id)


print("Number of topics (excluding outliers):", len(topic_docs))

Number of topics (excluding outliers): 5


In [10]:
docs_per_topic[0]

['It worked great for the first couple of weeks then it just stopped completely.. so basically a small waste of money.',
 'does not have the need amps to charge things like ipads, or hp touchpads. but its super small and compact.',
 "I am disappointed that the 1A didn't work with my iPad.  That's what I get for buying a cheap adapter.",
 'This is a nice charger but you can tell it was made cheaply in China.  When it is charging the phone, the car radio gets LOTS of static.  Not so much that I have to stop charging but like when you are near power lines and the radio station is far away.So, no RF shielding.I gave it 4 stars because it works fine for me, but if you listen to the radio, you might consider it is more like 2 or 3 stars.',
 'After a week only one side works',
 "Only works one side at a time. When you connect two cables, one side stop working and also overheated burning the fuses. I purchased two of them and it's the same problem. Cheap and bad quality.",
 "Didn't last very l

In [14]:
ctfidf_vectorizer = CTFIDFVectorizer(**config.c_tfidf_config)
c_tfidf_matrix, vocab = ctfidf_vectorizer.fit_transform(topic_docs)

(5, 939)


In [ ]:

        
top_n_words = 20
topic_words_ = {}

for i, topic_id in enumerate(topic_ids):
    topic_scores = c_tfidf_matrix[i]
    
    top_indices = topic_scores.argsort()[-top_n_words:][::-1]
    
    words = [vocab[j] for j in top_indices]
    scores = [topic_scores[j] for j in top_indices]
    
    topic_words_[topic_id] = list(zip(words, scores))

topics_ = {
    topic: [word for word, score in words]
    for topic, words in topic_words_.items()
}

{np.int64(0): ['slot',
  'charge',
  'charging',
  'cheap',
  'design',
  'worked',
  'works',
  'died',
  'slot design',
  'usb',
  'charger',
  'iphone',
  'small',
  'stars',
  'radio',
  'amps',
  'months',
  'power',
  'car',
  'bought'],
 np.int64(1): ['great',
  'wouldn',
  'work',
  'just',
  'travel',
  'helpful',
  'months',
  'little',
  've',
  'problem',
  'bought',
  'works',
  'problem yes',
  'correctly',
  'electrical product',
  'power say',
  'don forever',
  'correctly helpful',
  'don',
  'couple especially'],
 np.int64(2): ['good',
  'price',
  'product',
  'price good',
  'looking',
  'easy',
  'time',
  'great',
  'just',
  'works',
  'buy ask',
  'buy',
  'free',
  'come good',
  'condition',
  'condition thank',
  'come',
  'free shipping',
  'love griffin',
  'phone htc'],
 np.int64(3): ['blue',
  'clear',
  'plug',
  'blue light',
  'pros',
  'car',
  'love',
  'nice',
  'product',
  'light',
  'charge',
  'just',
  'night',
  'pros 95',
  'night overall',
 

In [ ]:
config. c_tfidf_config["use_bm25"] = True

ctfidf_vectorizer = CTFIDFVectorizer(**config.c_tfidf_config)
c_tfidf_matrix, vocab = ctfidf_vectorizer.fit_transform(topic_docs)

        
top_n_words = 20
topic_words_ = {}

for i, topic_id in enumerate(topic_ids):
    # Get scores for this topic
    topic_scores = c_tfidf_matrix[i]
    
    # Get indices of top N words (sorted descending)
    top_indices = topic_scores.argsort()[-top_n_words:][::-1]
    
    # Get words and their scores
    words = [vocab[j] for j in top_indices]
    scores = [topic_scores[j] for j in top_indices]
    
    topic_words_[topic_id] = list(zip(words, scores))

# Create a simple topics_ dictionary (just words, no scores)
topics_ = {
    topic: [word for word, score in words]
    for topic, words in topic_words_.items()
}

(5, 939)


{np.int64(0): ['slot',
  'charge',
  'charging',
  'cheap',
  'design',
  'worked',
  'works',
  'died',
  'slot design',
  'usb',
  'charger',
  'iphone',
  'small',
  'stars',
  'radio',
  'amps',
  'months',
  'power',
  'car',
  'bought'],
 np.int64(1): ['great',
  'wouldn',
  'work',
  'just',
  'travel',
  'helpful',
  'months',
  'little',
  've',
  'problem',
  'bought',
  'works',
  'problem yes',
  'correctly',
  'electrical product',
  'power say',
  'don forever',
  'correctly helpful',
  'don',
  'couple especially'],
 np.int64(2): ['good',
  'price',
  'product',
  'price good',
  'looking',
  'easy',
  'time',
  'great',
  'just',
  'works',
  'buy ask',
  'buy',
  'free',
  'come good',
  'condition',
  'condition thank',
  'come',
  'free shipping',
  'love griffin',
  'phone htc'],
 np.int64(3): ['blue',
  'clear',
  'plug',
  'blue light',
  'pros',
  'car',
  'love',
  'nice',
  'product',
  'light',
  'charge',
  'just',
  'night',
  'pros 95',
  'night overall',
 

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

def extract_keybert_keywords(topic_docs, embeddings, vocab, top_n=15):
    """Extract keywords using embedding similarity"""
    # Get embeddings for vocabulary words
    word_embeddings = embedder.embed(vocab)
    
    refined_topics = {}
    for topic_id, topic_doc in enumerate(topic_docs):
        # Get embedding for the concatenated topic document
        topic_embedding = embedder.embed([topic_doc])[0]
        
        # Calculate similarity between topic and all words
        similarities = cosine_similarity(
            [topic_embedding], 
            word_embeddings
        )[0]
        
        # Get top N most similar words
        top_indices = similarities.argsort()[-top_n:][::-1]
        refined_topics[topic_id] = [
            (vocab[i], similarities[i]) for i in top_indices
        ]
    
    return refined_topics

keybert_topics = extract_keybert_keywords(topic_docs, embeddings, vocab)

Using device: Apple Silicon GPU (MPS)
  macOS: True


Batches: 100%|██████████| 118/118 [00:01<00:00, 77.07it/s]


Using device: Apple Silicon GPU (MPS)
  macOS: True


Batches: 100%|██████████| 1/1 [00:00<00:00, 12.86it/s]


Using device: Apple Silicon GPU (MPS)
  macOS: True


...

Batches: 100%|██████████| 1/1 [00:00<00:00, 23.52it/s]


Using device: Apple Silicon GPU (MPS)
  macOS: True


Batches: 100%|██████████| 1/1 [00:00<00:00, 24.34it/s]


Using device: Apple Silicon GPU (MPS)
  macOS: True


Batches: 100%|██████████| 1/1 [00:00<00:00, 19.78it/s]


In [18]:
llm_topic_namer = LLMTopicNamer(**config.LLM_CONFIG)
topic_names = llm_topic_namer.name_topics(topics_)

INFO - Initializing LLM: Qwen/Qwen3-8B


Loading checkpoint shards: 100%|██████████| 5/5 [00:12<00:00,  2.56s/it]
'(ProtocolError('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')), '(Request ID: afdd573c-2548-4dea-b3ed-2a864ad14b04)')' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen3-8B/resolve/main/generation_config.json


WARNING - '(ProtocolError('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')), '(Request ID: afdd573c-2548-4dea-b3ed-2a864ad14b04)')' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen3-8B/resolve/main/generation_config.json


Retrying in 1s [Retry 1/5].


WARNING - Retrying in 1s [Retry 1/5].


...

WARNING - '(ProtocolError('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')), '(Request ID: 3a14d0d5-3ddd-4548-8e96-0570326356ee)')' thrown while requesting HEAD https://huggingface.co/api/resolve-cache/models/Qwen/Qwen3-8B/b968826d9c46dd6066d109eabc6255188de91218/generation_config.json


Retrying in 1s [Retry 1/5].


WARNING - Retrying in 1s [Retry 1/5].


Device set to use mps:0


INFO - LLM initialized successfully
Task: Create a short, accurate topic label from ranked keywords.
Write a natural-sounding noun phrase (not a keyword list).
Be specific when the keywords support it; do not invent details.
Avoid hype/marketing language. Avoid emojis.
If ambiguous, choose the broadest correct umbrella label.
Domain: Customer complaints.
Emphasize the issue/failure mode (e.g., billing problems, delivery delays, defects, account access, poor support).
Use professional, objective phrasing.


You will be given ranked keywords for ONE topic.
Return ONLY a valid JSON object (no markdown, no code fences, no extra text).

JSON schema (exact keys):
{"topic_name": "2-6 words", "reasoning": "one short sentence"}

Constraints:
- topic_name: 2 to 6 words, descriptive, no trailing period
- reasoning: ONE short sentence (<= 20 words) explaining why the name fits the keywords

Keywords (ranked): slot, charge, charging, cheap, design, worked, works, died, slot design, usb, charger, ip

In [19]:
topic_names

{np.int64(0): 'slot, charge, charging',
 np.int64(1): 'great, wouldn, work',
 np.int64(2): 'good, price, product',
 np.int64(3): 'blue, clear, plug',
 np.int64(4): 'great, charge, car'}

In [20]:
topic_names = llm_topic_namer.name_topics(keybert_topics)

Task: Create a short, accurate topic label from ranked keywords.
Write a natural-sounding noun phrase (not a keyword list).
Be specific when the keywords support it; do not invent details.
Avoid hype/marketing language. Avoid emojis.
If ambiguous, choose the broadest correct umbrella label.
Domain: Customer complaints.
Emphasize the issue/failure mode (e.g., billing problems, delivery delays, defects, account access, poor support).
Use professional, objective phrasing.


You will be given ranked keywords for ONE topic.
Return ONLY a valid JSON object (no markdown, no code fences, no extra text).

JSON schema (exact keys):
{"topic_name": "2-6 words", "reasoning": "one short sentence"}

Constraints:
- topic_name: 2 to 6 words, descriptive, no trailing period
- reasoning: ONE short sentence (<= 20 words) explaining why the name fits the keywords

Keywords (ranked): cheap small, cheap adapter, worked cheap, simply disposable, smaller charger, cheap bad, cheaply china, car charger, maybe ch

In [21]:
topic_names

{0: 'cheap small, cheap adapter, worked cheap',
 1: 'surprisingly inexpensive, excellent value, works iphone',
 2: 'works iphone, nice charger, car charger',
 3: 'cables pretty, led glows, charge devices',
 4: 'sleek convenient, car charger, charger love'}

In [ ]:
from src.model.visualize import TopicVisualizer

vis = TopicVisualizer(template="plotly_white")

fig_scatter = vis.plot_embeddings_2d(
    embeddings=reduced_embeddings,          # shape (n_docs, >=2)
    labels=clusters_labels,                # shape (n_docs,)
    hover_text=docs,                       # optional: show doc snippets on hover
    title="Reduced embeddings (first 2 dims) colored by topic",
    random_state=config.SEED,
)
fig_scatter.show()


fig_dist = vis.plot_topic_distribution(
    labels=clusters_labels,
    top_n=25,
    title="Topic distribution (top 25, excluding outliers)",
)
fig_dist.show()

fig_grid = vis.plot_topics_words_grid(
    topic_words=topic_words_,
    n_topics=9,
    n_words=8,
    n_cols=3,
    title="Top words per topic (overview)",
)
fig_grid.show()


labels_np = np.asarray(clusters_labels)
unique, counts = np.unique(labels_np[labels_np >= 0], return_counts=True)
top_topic_id = int(unique[np.argmax(counts)]) if len(unique) else 0

fig_topic = vis.plot_topic_words(
    topic_words=topic_words_,
    topic_id=top_topic_id,
    n_words=15,
    title=f"Topic {top_topic_id}: top words",
)
fig_topic.show()